In [1]:
!pip install selenium
!pip install webdriver

ERROR: Could not find a version that satisfies the requirement webdriver (from versions: none)
ERROR: No matching distribution found for webdriver


In [2]:
import csv 
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import os
import sys
os.path.dirname(sys.executable)
from time import sleep
from selenium import webdriver
from selenium.webdriver import Chrome 
from selenium.webdriver.chrome.options import Options as ChromeOptions
browser = webdriver.Chrome()
driver = webdriver.Chrome()

In [10]:
def get_tweet_data(card):
    """Extract data from tweet card"""
    username = card.find_element("xpath",'.//span').text
    try:
        handle = card.find_element("xpath",'.//span[contains(text(), "@")]').text
    except NoSuchElementException:
        return
    
    try:
        postdate = card.find_element("xpath",'.//time').get_attribute('datetime')
    except NoSuchElementException:
        return
    
    comment = card.find_element("xpath",'.//div[2]/div[2]/div[1]').text
    responding = card.find_element("xpath",'.//div[2]/div[2]/div[2]').text
    text = comment + responding
    reply_cnt = card.find_element("xpath",'.//div[@data-testid="reply"]').text
    retweet_cnt = card.find_element("xpath",'.//div[@data-testid="retweet"]').text
    like_cnt = card.find_element("xpath",'.//div[@data-testid="like"]').text

    
    tweet = (username, handle, postdate, text, reply_cnt, retweet_cnt, like_cnt)
    return tweet  

In [11]:
#log in to twitter
driver.get('https://twitter.com/login')

In [12]:
search_term = input('search term: ')

#driver.get('https://twitter.com/search')
driver.maximize_window()
sleep(5)

# find search input and search for term
search_input = driver.find_element("xpath",'//input[@aria-label="Search query"]')
search_input.send_keys(search_term)
search_input.send_keys(Keys.RETURN)
sleep(1)

# navigate to historical 'latest' tab
driver.find_element_by_link_text('Latest').click()

search term: 蔣萬安


AttributeError: 'WebDriver' object has no attribute 'find_element_by_link_text'

In [13]:
# get all tweets on the page
data = []
tweet_ids = set()
last_position = driver.execute_script("return window.pageYOffset;")
scrolling = True

while scrolling:
    page_cards = page_cards = driver.find_elements("xpath", '//article[@data-testid="tweet"]')

    #driver.find_elements_by_xpath('//article[@data-testid="tweet"]')
    for card in page_cards[-15:]:
        tweet = get_tweet_data(card)
        if tweet:
            tweet_id = ''.join(tweet)
            if tweet_id not in tweet_ids:
                tweet_ids.add(tweet_id)
                data.append(tweet)
            
    scroll_attempt = 0
    while True:
        # check scroll position
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        sleep(2)
        curr_position = driver.execute_script("return window.pageYOffset;")
        if last_position == curr_position:
            scroll_attempt += 1
            
            # end of scroll region
            if scroll_attempt >= 4:
                scrolling = False
                break
            else:
                sleep(2) # attempt another scroll
        else:
            last_position = curr_position
            break

# close the web driver
driver.close()

In [14]:
data

[('Asian American',
  '@AsianAmericanUS',
  '2022-11-15T04:56:37.000Z',
  'Asian American\n@AsianAmericanUS\n·\nNov 15, 2022Replying to \n@TaipeiShihChung\n說真的，#蔣萬安 無能嗎？他最能幹的地方就是跟蔣家聯繫。那個 #黃珊珊 有全台的老軍給她靠山。 #陳時中 呢？只在這三年防禦這一方面保護台灣人，讓台灣在全世界變成最優秀的國家。只有這樣子啊！大不了！\n6',
  '',
  '',
  '6'),
 ('Asian American',
  '@AsianAmericanUS',
  '2022-11-14T04:31:35.000Z',
  'Asian American\n@AsianAmericanUS\n·\nNov 14, 2022Replying to \n@kuomintang\n and \n@eric_chu0607\nThat’s funny, because we all know that the KMT #中國 #國民黨 has made ZERO contribution to Taiwan’s democracy. Not Eric Chu #朱立倫, not Ma Ying-Jeou #馬英九 and definitely NOT the Chiang family #蔣家 。國民黨對台灣的民主化一點都沒有貢獻。 #蔣萬安\n2\n5\n25',
  '2',
  '5',
  '25'),
 ('觔斗雲 #FuckPutin #StandWithUkraine',
  '@Saltocloud',
  '2022-11-13T15:44:11.000Z',
  '觔斗雲 #FuckPutin #StandWithUkraine\n@Saltocloud\n·\nNov 13, 2022@WananChiang\n 欸萬安啊～不得不說，你的中文，真他媽爛透了\n\n#蔣萬安 #章萬安 #x萬安 \n#笑CRY',
  '',
  '',
  ''),
 ('長輩圖',
  '@TISAtoromi',
  '2022-11-13T05:40:06.000Z',
  '長輩圖\n

In [16]:
with open('extrajiang.csv', 'w', newline='', encoding='utf-8') as f:
    header = ['UserName', 'Handle', 'Timestamp', 'Text', 'Comments', 'Likes', 'Retweets']
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(data)